# Ensemble learning
#### Author: Matteo Caorsi

It is possible, in giotto-deep, to use the ensemble models of the library `ensemble-pytorch` together with all the functionalities of `gdeep`.

## Scope

Ensamble technique put together the predictions of different models and decide which is the best answer. It is a bit like having and ensemble of experts giving opinions and then the person in charge takes the final decision. In this example, we will try out the `VotingClassifer`, i.e. an ensemble method that decides on the best preediciton based on the majority of experts votes.

## Content
These aree the main steps we wll follow:
 1. Load your data
 2. Defne a single expert
 3. wrap the ensemble model
 4. train the ensemble

In [ ]:
# imports
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ExponentialLR
from torch.optim import SGD, Adam
from torchensemble import VotingClassifier

from gdeep.data.datasets import DatasetBuilder, DataLoaderBuilder
from gdeep.trainer import Trainer
from gdeep.utility.optimisation import SAMOptimizer
from gdeep.models import FFNet
from gdeep.utility import ensemble_wrapper
from gdeep.visualisation import Visualiser


# Initialize the tensorboard writer

In order to analyse the results of your models, you need to start tensorboard.
On the terminal, move inside the `/examples` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training to see all the visualisation results.

In [ ]:
writer = SummaryWriter()


# Load your data

In this example we use a tabular dataset and the task is a classification task. The dataset is a point cloud representing two entangled tori and the model needs to classify each point as belonging to one or the other torus.

In [ ]:
bd = DatasetBuilder(name="DoubleTori")
ds_tr, ds_val, _ = bd.build()
# train_indices = list(range(160))
dl = DataLoaderBuilder((ds_tr, ds_val))
dl_tr, dl_val, dl_ts = dl.build(({"batch_size": 23}, {"batch_size": 23}))


# Define a single estimator of the ensemble

You can define a single estimator of an ensemble as you would normally do with any other neural network

In [ ]:
# a simple model
class model1(nn.Module):
    def __init__(self):
        super(model1, self).__init__()
        self.seqmodel = nn.Sequential(nn.Flatten(), FFNet(arch=[3, 7, 7, 2]))

    def forward(self, x):
        return self.seqmodel(x)


# Wrap the ensamble models

You can wrap the ensamble-pytorch models with the utiliy function `ensemble_wrapper`. Here below the concrete example:

In [ ]:
model = ensemble_wrapper(VotingClassifier)(
    estimator=model1, n_estimators=10, cuda=False
)


What you would have done instead, renouncing to many giotto-deep capablities:

```
model = VotingClassifier(
    estimator=model1,
    n_estimators=10,
    cuda=False
)
model.set_optimizer("Adam")
model.fit(train_loader=dl_tr,epochs=1)
```


# Train your ensemble of models
You can easily train your ensemble model as you would train any other model in giotto-deep: initialise the `Trainer` class and run it.

In [ ]:
# initlaise the loss function
loss_fn = nn.CrossEntropyLoss()
# initialise the pipelien class
pipe = Trainer(model, (dl_tr, dl_ts), loss_fn, writer)

# initialise the SAM optimiser
optim = SAMOptimizer(SGD)  # this is a class, not an instance!

# train the model with learning rate scheduler
pipe.train(
    optim,
    7,
    False,
    optimizers_param={"lr": 0.01},
    profiling=False,
    store_grad_layer_hist=True,
    writer_tag="ensemble",
)


# Visualise the model graph

You can integractively visualise your ensemble of models by checking it on tensorboard after these few lines are executed:

In [ ]:
# initialise he visualiser
vs = Visualiser(pipe)

# send the graph of the model to tensorboard
vs.plot_data_model()
